In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import logging
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# read-in cleaned data and parse to corpus and tokenizer
# path = '../data/'
path = '/content/drive/MyDrive/Colab Notebooks/'
filepath = path + 'allphishsets.csv'

df = pd.read_csv(filepath)
df = df.sort_values(by=['showdate', 'set', 'position'],
                    ascending=[True, True, True])

df.loc[df['times_played'] <= 2, 'slug'] = 'wildcard'
df.loc[df['times_played'] <= 2, 'times_played'] = 510

songstring = df[['showdate', 'set', 'slug']].groupby(['showdate', 'set'])['slug']\
                                            .apply(lambda x: '|'.join(x)).reset_index()

songstring['full'] = songstring.apply(lambda row: f"set-{row['set']}|{row['slug']}", axis=1)

songstring = songstring[['showdate', 'full']].groupby(['showdate'])['full']\
                                             .apply(lambda x: '|'.join(x)).reset_index()

songstring['full'] += '|eos'

songstring = songstring[songstring['showdate']>'1990-01-01']

corpus = [''.join(map(lambda s: s.replace('|', ' '), f))
          for f in songstring['full']]

corpus = ' '.join(corpus).split(' ')

def PrepareDataset(corpus: list, n: int,
                   batch_size: int, train_split: float):
    """
    Prepares Datasets for training and validation data from Setlist data
    Args:
      corpus :: list :: full corpus of songs composed of song sequences
      n :: int :: sequence length to trim
      batch_size :: int :: batch size for datasets
      train_split :: float :: values between 0 and 1, splits the data for
                              training and validation
    """
    texts = []
    for i in range(len(corpus)-n):
        texts.append(' '.join(corpus[i:i+n]))

    tokenizer = Tokenizer(filters='')
    tokenizer.fit_on_texts(texts)

    x_inputs = []
    x_outputs = []
    for line in texts:
        token_list = tokenizer.texts_to_sequences([line])[0]
        x_inputs.append(token_list[:-1])  #drop last song
        x_outputs.append(token_list[1:])  #drop first song

    x_inputs = np.array(x_inputs)
    x_outputs = np.array(x_outputs)

    buffer_size = len(x_inputs)
    train_size = int(train_split*buffer_size)

    dataset = tf.data.Dataset.from_tensor_slices((x_inputs, x_outputs))
    shuffled_data = dataset.shuffle(buffer_size)

    train_data = dataset.take(train_size) \
                        .batch(batch_size) \
                        .prefetch(buffer_size=tf.data.AUTOTUNE)

    val_data = dataset.skip(train_size) \
                      .batch(batch_size) \
                      .prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_data, val_data, tokenizer

In [3]:
# https://www.tensorflow.org/text/tutorials/transformer
# classes for positional embedding and attention layers
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)


class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x


class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()


class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [4]:
# classes for NN model architecture
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x


class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, *, d_model, num_heads, dff, dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    y = self.causal_self_attention(x=x)

    x = self.add([x, y])
    x = self.layer_norm(x)

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x


class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

  def call(self, x):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x)

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x


class GPT(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, x):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.

    x = self.decoder(x)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    # Drop the keras mask, so it doesn't scale the losses/metrics.
    try:
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits


class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


# performance metrics
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [5]:
train_data, val_data, tokenizer= PrepareDataset(
    corpus=corpus,
    n=100,
    batch_size=32,
    train_split=0.8
)

unique_words = len(tokenizer.word_index) + 1
d_model = 128

learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(
    learning_rate,
    beta_1=0.9,
    beta_2=0.98,
    epsilon=1e-9
)

num_layers = 8
dff = 512
num_heads = 8
dropout_rate = 0.1
epochs = 10

gpt = GPT(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=unique_words,
    target_vocab_size=unique_words,
    dropout_rate=dropout_rate
)

gpt.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy]
)

gpt.fit(train_data, validation_data=val_data, epochs=epochs)

Epoch 1/10
963/963 [==============================] - 162s 129ms/step - loss: 4.6570 - masked_accuracy: 0.1512 - val_loss: 4.9318 - val_masked_accuracy: 0.1712
Epoch 2/10
963/963 [==============================] - 115s 119ms/step - loss: 3.8084 - masked_accuracy: 0.2287 - val_loss: 5.1667 - val_masked_accuracy: 0.1693
Epoch 3/10
963/963 [==============================] - 116s 121ms/step - loss: 3.7265 - masked_accuracy: 0.2316 - val_loss: 5.3794 - val_masked_accuracy: 0.1608
Epoch 4/10
963/963 [==============================] - 115s 119ms/step - loss: 3.7474 - masked_accuracy: 0.2308 - val_loss: 5.3153 - val_masked_accuracy: 0.1670
Epoch 5/10
963/963 [==============================] - 123s 128ms/step - loss: 3.7354 - masked_accuracy: 0.2327 - val_loss: 5.2209 - val_masked_accuracy: 0.1756
Epoch 6/10
963/963 [==============================] - 115s 119ms/step - loss: 3.6876 - masked_accuracy: 0.2374 - val_loss: 5.2473 - val_masked_accuracy: 0.1671
Epoch 7/10
963/963 [====================

In [6]:
# model the output
test_idx = 1

for input, labels in val_data.take(test_idx):
    break

test_labels = labels[0][12:-4]

output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)

for i in range(len(test_labels.numpy())):
    output_array = output_array.write(i, test_labels.numpy()[i])


for i in tf.range(len(input[0])):
    output = tf.reshape(output_array.stack(), (1, len(output_array.stack())))
    predictions = gpt(output, training=False)

    # Select the last token from the `seq_len` dimension.
    predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

    predicted_id = tf.argmax(predictions, axis=-1)[0][0]

    # Concatenate the `predicted_id` to the output which is given to the
    # decoder as its input.
    output_array = output_array.write(i+1, predicted_id)

    if predicted_id == 4:
        break

[tokenizer.index_word[s] for s in output_array.stack().numpy()]

['set-1',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 'acdc-bag',
 '

In [ ]:
[tokenizer.index_word[s] for s in labels[test_idx-1].numpy() if s != 0]

NameError: ignored

In [ ]:
[tokenizer.index_word[s] for s in encoder_input.numpy()[0] if s != 0]

['set-1',
 'fuego',
 'my-soul',
 'back-on-the-train',
 '555',
 'dog-faced-boy',
 'fuck-your-face',
 'horn',
 'frankie-says',
 'my-friend-my-friend',
 'roses-are-free',
 'roggae',
 'birds-of-a-feather',
 'wingsuit',
 'set-2',
 'possum',
 'crosseyed-and-painless',
 'light',
 'the-dogs',
 'lengthwise',
 'twist',
 'wading-in-the-velvet-sea',
 'harry-hood',
 'golgi-apparatus',
 'backwards-down-the-number-line',
 'set-e',
 'waiting-all-night',
 'sing-monica',
 'the-star-spangled-banner',
 'eos']